In [3]:
# 回顾感知机
# nn.Sequential 定义了一种特殊的Module
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)  # 创建一个形状为(2,20)的张量，其中的元素是均匀分布中随机抽取，范围 0-1
X, net(X)

(tensor([[0.8749, 0.2246, 0.7873, 0.3886, 0.1643, 0.0534, 0.6975, 0.4991, 0.4113,
          0.5312, 0.9499, 0.8090, 0.7174, 0.6089, 0.6567, 0.7868, 0.6155, 0.2169,
          0.7337, 0.2052],
         [0.4667, 0.7871, 0.5516, 0.9840, 0.2385, 0.4015, 0.8081, 0.5515, 0.4905,
          0.8857, 0.6735, 0.7608, 0.3855, 0.7809, 0.2398, 0.3384, 0.6985, 0.9164,
          0.0798, 0.1506]]),
 tensor([[-0.0763,  0.2052, -0.0221, -0.0063,  0.0793, -0.0930, -0.0534,  0.4438,
          -0.2552, -0.2870],
         [-0.0064,  0.1052, -0.1009, -0.0029,  0.0116, -0.1599, -0.0462,  0.4368,
          -0.1410, -0.2636]], grad_fn=<AddmmBackward0>))

每个块必须提供的基本功能
1. 将输入数据作为其向前传播函数的参数
2. 通过前向传播函数来生成输出
3. 计算其输出关于输入的梯度，可通过其反向传播函数进行访问，通常自动发生
4. 存储和访问向前传播计算所需的参数
5. 根据需要初始化模型参数

In [4]:
# 1.自定义块
class MLP(nn.Module):
    # 用模型参数声明层，这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)     # 输出层
    
    # 定义模型的前向传播，即如何根据输入 X 返回所需的模型输出
    def forward(self, X):
        # 注意：这里我们使用ReLU函数版本，其在nn.functional 模块中定义
        return self.out(F.relu(self.hidden(X)))

In [5]:
# 实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层
net = MLP()
net(X)

tensor([[-0.0595, -0.0430,  0.0897,  0.0092,  0.0445, -0.1018,  0.0749, -0.0186,
          0.0205,  0.0082],
        [ 0.0164,  0.0198,  0.0947, -0.0214,  0.0125, -0.0801,  0.1584,  0.0141,
          0.0911, -0.0122]], grad_fn=<AddmmBackward0>)

现在我们可以更仔细地看看Sequential类是如何工作的， 回想一下Sequential的设计是为了把其他模块串起来。 为了构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：

1. 一种将块逐个追加到列表中的函数；

2. 一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

下面的MySequential类提供了与默认Sequential类相同的功能。

In [7]:
# 2. 顺序块
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_module中，_module的类型是OrderedDict
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict 保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

\_init_ 函数将每个模块逐个添加到有序字典_modules中。 
\_modules的主要优点是： 在模块的参数初始化过程中， 系统知道在_modules字典中查找需要初始化参数的子块。

当MySequential的前向传播函数被调用时， 每个添加的块都按照它们被添加的顺序执行。 现在可以使用我们的MySequential类重新实现多层感知机

In [8]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0784, -0.0997,  0.0847, -0.3399,  0.3741,  0.0969,  0.2020, -0.1440,
          0.0521, -0.0545],
        [-0.1515, -0.1083, -0.0465, -0.3770,  0.2863,  0.1587,  0.1655, -0.1569,
          0.1741, -0.1354]], grad_fn=<AddmmBackward0>)

Sequential类使模型构造变得简单，允许我们组合新的架构，而不必定义自己的类。然而，并不是所有的架构都是简单的
顺序架构。当需要更强的灵活性时，我们需要定义自己的块。例如，我们可能希望在前向传播函数中执行Python的控制流
此外，我们可能希望执行任意的数学运算，而不是简单地依赖预定义的神经网络层。
到目前为止，我们网络中的所有操作都对网络的激活值及网络的参数起作用。然而，有时我们可能希望合并既不是上一层的
结果也不是可更新参数的项，我们称之为常数参数（constant parameter）。例如，我们需要一个计算函数
 $f({\bf x},{\bf w})=c\,.$ w x的层，其中x是输入，w是参数，c是某个在优化过程中没有更新的指定常量。因此我们实现了一个
F ixedHi ddenML P类，如下所示;

In [11]:
#  3.在前向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度随机权重参数，因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层，这相对于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.2614, grad_fn=<SumBackward0>)

在这个FixedHiddenMLP模型中，我们实现了一个隐藏层，其权重（self.rand_weight）在实例化时被随机初始化，之
后为常量。这个权重不是一个模型参数，因此它永远不会被反向传播更新。然后，神经网络将这个固定层的输出通过一个全
连接层。
注意，在返回输出之前，模型做了一些不寻常的事情：它运行了一个while循环，在Li范数大于1的条件下，将输出向量除以
2，直到它满足条件为止。最后，模型返回了X中所有项的和。注意，此操作可能不会常用于在任何实际任务中，我们只展
示如何将任意代码集成到神经网络计算的流程中

In [13]:
# 混合搭配各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64,32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0051, grad_fn=<SumBackward0>)